# Rossman dataset 

Fuente xgboost: https://xgboost.readthedocs.io/en/release_3.0.0/

https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.ensemble.XGBRegressor

Correr esto:

!pip install xgboost==1.7.6 scikit-learn==1.2.2

https://www.kaggle.com/competitions/rossmann-store-sales/data

- Id - an Id that represents a (Store, Date) duple within the test set
- Store - a unique Id for each store
- Sales - the turnover for any given day (this is what you are predicting)
- Customers - the number of customers on a given day
- Open - an indicator for whether the store was open: 0 = closed, 1 = open
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
- StoreType - differentiates between 4 different store models: a, b, c, d
- Assortment - describes an assortment level: a = basic, b = extra, c = extended
- CompetitionDistance - distance in meters to the nearest competitor store
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- Promo - indicates whether a store is running a promo on that day
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# Carga de datos
train = pd.read_csv('train.csv')
stores = pd.read_csv('store.csv')

# Merge de ambos datasets

# Convertir la columna de fecha

# Ordenar por fecha para evitar leakage

# Filtrar solo tiendas abiertas

# Variables temporales útiles, mes y dia del mes

# df['week_of_year'] = df['Date'].dt.isocalendar().week.astype(int)

# Eliminar columnas que no usaremos


/var/folders/q7/z6bgw1fj4vng6rs97g1chhz00000gn/T/ipykernel_83435/3565455745.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('rossman.csv')


In [94]:
# split 90% de datos

# Obten X_train, y_train, X_test, y_test

# Codificar variables categóricas (después del split para evitar leakage)

# Alinear columnas en test con train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [96]:
# Xgboost

# Fit

# Predicción y evaluación


R²: 0.5352


In [2]:
# Crea una regresion lineal para ver que tan bien da
from sklearn.linear_model import LinearRegression

# Calcular medias solo en train


# Aplicar a ambos

# Entrenar modelo

# Predecir

# Evaluar


### Tuneando xgboost

- n_estimators: número total de árboles a entrenar. Si usas early_stopping_rounds, puedes poner un número alto sin preocuparte por overfitting.

- max_depth: profundidad máxima de cada árbol. Profundidades mayores capturan más complejidad, pero pueden sobreajustar.

- learning_rate: qué tan rápido aprende el modelo. Valores más bajos requieren más árboles (n_estimators más alto), pero suelen generalizar mejor.

- tree_method: método para construir árboles:

    'exact': busca el mejor split evaluando todos los valores.
    
    'hist': agrupa en bins (default moderno), mucho más rápido.
    
    'gpu_hist': como 'hist' pero usando GPU (⚡ si tienes una buena GPU).

- subsample: fracción de los datos usados en cada árbol (filas). Ayuda a reducir overfitting. Ej: 0.8 usa el 80% de los datos en cada iteración.

- colsample_bytree (¡ojo con el nombre!): fracción de columnas seleccionadas aleatoriamente para cada árbol. También ayuda contra el overfitting.

- early_stopping_rounds: si la métrica en el conjunto de validación no mejora durante N iteraciones, se detiene el entrenamiento (requiere eval_set).

- eval_metric: métrica usada para evaluar el desempeño durante entrenamiento (ej. 'rmse', 'mae', 'logloss', 'auc'). Es la que define si se activa early_stopping.

- gamma: mínima ganancia necesaria para hacer un split adicional. Ayuda a regular complejidad.

- reg_lambda: regularización L2 (Ridge). Útil para evitar sobreajuste.

- reg_alpha: regularización L1 (Lasso). Puede hacer pruning de features.



In [3]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Crea el modelo

# Haz fit, usa early stopping, verboso y evalset [(X_train, y_train), (X_test, y_test)]

# Extraer resultados después del fit, model.evals_result()

# Predicción y métricas


In [4]:
# Grafica ambos errores 


In [7]:
# Calibracion

### Feature importances 

In [ ]:
#

```
from sklearn.inspection import PartialDependenceDisplay

for feature in importancias.features:
    PartialDependenceDisplay.from_estimator(model, X_test, [feature])
    plt.show()
```

In [5]:
# Partial dependence plot 


# Feature importance

### Gain 
Gain es que tanto disminuye la función de perdida, en promedio al usar cada feature


In [6]:
from xgboost import plot_importance

# Grafica la importancia, por importance_type y el max_num_features


### Weight
Weighr es el porcentaje de veces que se uso esa variable